# Ejemplo de proceso Data Science #2: Deteccion de contenido adulto en paginas web.


En este ejemplo creamos una aplicacion, que dada una url, nos dirá si dicha url es de contenido para adultos o no.


Usaremos Scrapy para scrapear el dataset de entrenamiento, NTLK para el procesado de lenguaje natural y Scikit-Learn para el Machine Learning

## Definición del Problema


Una empresa de ropa de bebés ha gastado una cantidad no considerable en anuncios online. Sin embargo, ha recibido noticias de que sus anuncios se han mostrado en páginas de contenido adulto. El CEO quiere analizar los datos históricos de anuncios mostrados y asegurarse de que no se han mostrado sus anuncios en páginas adultas 

# 1. Obtención del dataset de entrenamiento.

Crearemos una [araña](https://github.com/manugarri/jornada_data_science_info_2017/blob/master/ejemplos/Ejemplo2/spider.py) a la que le daremos un listado de links de agregadores de contenido adulto por una parte, y otro listado de links de contenido no adulto. Dicha araña simplemente parseara todo el HTML de cada link en dichas páginas.

# Leemos el dataset con los contenidos adultos y no adultos

In [24]:
import json
import pandas as pd

with open("data.jsonlines") as fname:
    lines = fname.readlines()
    rows = [json.loads(l) for l in lines]
    data = pd.DataFrame(rows)

# El dataset inicial contiene la url, el contenido html y una etiqueta que es 1 si el contenido es adulto y un 0 si no lo es

In [25]:
data.head(20)

,label,text,url
0,1,User:\n Pass:\n \n \n\n \n ...,http://www.kuntfutube.com/videos/all/date-0/
1,1,Home | \n Categ...,http://www.pk5.net/
2,1,youpunish.com\tEnlaces relacionadosEste domini...,http://ww1.youpunish.com
3,1,FuckingMonth Filter by post typeAll postsTextP...,http://g-o-z-a-r.tumblr.com/archive/
4,1,"(function() { document.write('<iframe alt="""" a...",http://www.yobt.tv/
5,1,SexgifMonth Filter by post typeAll postsTextPh...,http://sexgif-net.tumblr.com/archive/
6,1,phantomPopunder = (function () {\r\n\tvar phan...,http://vintage-erotica-forum.com/
7,1,Sign-In or Join\r\n\t\t \r\n\t \...,http://www.icanhazchat.com/welcome?gonewild
8,1,sonnbXenGalleryEnableInterlace = 0;\n\n\n\n ...,http://forum.xnxx.com/
9,1,Forumophilia\n\n\n\n\n\nRegister \n\nLog in ...,http://www.forumophilia.com


In [26]:
data.tail(20)

,label,text,url
833,0,2016 \n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,http://www.rawstory.com/
834,0,Log in / Sign upFollow JezebelFollowing Jezebe...,http://jezebel.com/
835,0,Press enter to search\n\t\t\t\t\t\t\t\t\t\t\n\...,http://www.feld.com/
836,0,var w_width = window.document.width || documen...,http://www.momtrends.com/
837,0,Join\n | \n Sign In...,http://www.modcloth.com/
838,0,"(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({...",http://weheartit.com/
839,0,Please upgrade your browser for the best Refin...,http://www.refinery29.com/
840,0,not-post\n\n\n\n\n\n \n\n\n\nHome\nAbout\nCont...,http://www.primermagazine.com/
841,0,.lt-ie10 .messenger.suggestions {\n dis...,http://www.nytimes.com/section/upshot
842,0,"Well Spent. - Obtainable, honestly crafted goo...",http://well-spent.com/


# Extraemos informacion de las tags HTML relevantes

In [36]:
from parsel import Selector

def parse_url(url_text):
    sel = Selector(text=url_text, type="html")
    meta_tags = ['keywords', 'description', 'category', 'RATING']
    tags = []
    for tag in meta_tags:
        tags.append( sel.xpath('//meta[@name="{}"]/@content'.format(tag)).extract_first())
    text = ''.join(sel.xpath("//body//text()").extract()).strip()
    text += ' '.join([i for i in tags if i])
    return text

In [37]:
data['all_text'] = data["text"].apply(parse_url)

In [38]:
data.head()

,label,text,url,all_text
0,1,User:\n Pass:\n \n \n\n \n ...,http://www.kuntfutube.com/videos/all/date-0/,User:\n Pass:\n \n \n\n \n ...
1,1,Home | \n Categ...,http://www.pk5.net/,Home | \n Categ...
2,1,youpunish.com\tEnlaces relacionadosEste domini...,http://ww1.youpunish.com,youpunish.com\tEnlaces relacionadosEste domini...
3,1,FuckingMonth Filter by post typeAll postsTextP...,http://g-o-z-a-r.tumblr.com/archive/,FuckingMonth Filter by post typeAll postsTextP...
4,1,"(function() { document.write('<iframe alt="""" a...",http://www.yobt.tv/,(function() { document.write('');})();\n \t...


# Usamos Procesado de Lenguaje Natural para limpiar el texto html de las paginas web y extraer las palabras relevantes

In [6]:
import string
import nltk
from nltk import stem, word_tokenize
import string
import numpy
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

stemmer = stem.PorterStemmer()
stemmer = stem.SnowballStemmer("english")

NLTK_STOPWORDS = stopwords.words('english')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems
"""
def normalize(s):
    if not s: return ''
    words = tokenize.wordpunct_tokenize(s.lower().strip())
    return [stemmer.stem(w) for w in words]
    return ' '.join([stemmer.stem(w) for w in words])
"""

[nltk_data] Downloading package stopwords to /home/manuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/manuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"\ndef normalize(s):\n    if not s: return ''\n    words = tokenize.wordpunct_tokenize(s.lower().strip())\n    return [stemmer.stem(w) for w in words]\n    return ' '.join([stemmer.stem(w) for w in words])\n"

# Creamos un CountVectorizer, que convierte una lista de palabras en una matriz de las palabras mas comunes y el numero de veces que aparecen en cada linea

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(strip_accents='unicode',ngram_range=(1,3), stop_words='english', tokenizer=tokenize)

# El output de vectorizer tiene este aspecto

In [10]:
text_vector = vectorizer.fit_transform(data.text)

In [14]:
text_vector.todense()

matrix([[15,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ..., 
        [ 3,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

# Creamos un pipeline (basicamente una lista de pasos que aplicar al dataset) con el countVectorizer y el Algoritmo SVC

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
pipeline = Pipeline([("vectorizer", vectorizer), ("svm", SVC(probability=True, class_weight='balanced'))])

# Entrenamos el modelo

In [16]:
pipeline_fit = pipeline.fit(data.text, data.label)

# Creamos funcion para detectar páginas adultas

In [17]:
from urllib.parse import urljoin
import requests
from urllib.parse import urlparse

def get_url_text(url):
    request = requests.get(url)
    return request.text

def detect_adult_page(url):
    url_text = get_url_text(url)
    data = parse_url(url_text)
    probs =  pipeline_fit.predict_proba([data])[0]
    print("Probabilities for url {}: Not Adult:{}; Adult:{}".format(url, probs[0], probs[1]))

# Testeamos

In [19]:
print("Paginas de contenido adulto")
detect_adult_page("http://www.redtube.com")
detect_adult_page("https://www.xvideos.com")

print("\n\nPaginas no de contenido adulto")
detect_adult_page("https://www.reddit.com")
detect_adult_page("http://nonhabitualresidents.com/#intro")
detect_adult_page("https://medium.com/@ericfeldman93/my-nephew-vs-ml-9e4519af499a")

Paginas de contenido adulto
Probabilities for url http://www.redtube.com: Not Adult:2.4783170527108597e-07; Adult:0.9999997521682947
Probabilities for url https://www.xvideos.com: Not Adult:0.15242570989439025; Adult:0.8475742901056097


Paginas no de contenido adulto
Probabilities for url https://www.reddit.com: Not Adult:0.7763417114404033; Adult:0.2236582885595967
Probabilities for url http://nonhabitualresidents.com/#intro: Not Adult:0.937109801951615; Adult:0.0628901980483849
Probabilities for url https://medium.com/@ericfeldman93/my-nephew-vs-ml-9e4519af499a: Not Adult:0.8478631482392281; Adult:0.15213685176077182
